### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

### Drop Null target records

In [4]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [5]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [6]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [7]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [8]:
features_list = [
# banking
'avg_daily_balance_last_1_months',
'max_balance_M1',
'min_balance_last_1_months',
'min_balance_M1',
'balance_5th_day_M2',
'balance_3th_day_M1',
'bounced_transactions_count',
'days_balance_lt_5000_M1',
'days_balance_lt_10000_M1',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',

'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

'cash_withdrawal_sum',
'num_debit_card_trans_last_2_months',
'ratio_active_loan_vs_salary',


    
#new banking features
'ratio_balance_after_2_days_max_credit_M1',
'ratio_balance_after_7_days_max_credit_M2',
'diff_btw_stated_salary_vs_actual_salary',
'ratio_5th_day_balance_to_max_credit_M3',
                 

'month_year',
'60_in_15mob']

In [9]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [10]:
oot_df = oot_df1[features_list]

In [11]:
model_df.shape, oot_df.shape

((101984, 23), (17343, 23))

In [12]:
model_df['month_year'].isnull().sum()

np.int64(0)

In [13]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3035332/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3035332/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [14]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3035332/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3035332/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [15]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [16]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [17]:
#Takeout record where banking data only present(exclude -9999)
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [18]:
model_df.shape

(4252, 23)

In [19]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]

In [20]:
oot_df.shape

(689, 23)

In [21]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [22]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [23]:
X_oot.shape,y_oot.shape

((689, 21), (689,))

In [24]:
X_model.shape

(4252, 21)

### Split Model dataset into Train and Test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [26]:
X_train.shape, X_test.shape

((2551, 21), (1701, 21))

In [27]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(2040, 21) (511, 21) (1701, 21)


In [28]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [29]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 5.2,
    'alpha': 4.5,
    'gamma': 4.8,
   
    'scale_pos_weight':4.0,
    'random_state': 42
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.64694	eval-auc:0.60698
[10]	train-auc:0.69308	eval-auc:0.62853
[20]	train-auc:0.70851	eval-auc:0.63672
[30]	train-auc:0.70762	eval-auc:0.63152
[40]	train-auc:0.71017	eval-auc:0.63695
[50]	train-auc:0.71300	eval-auc:0.63314
[60]	train-auc:0.71151	eval-auc:0.63413
[70]	train-auc:0.71223	eval-auc:0.63533
[72]	train-auc:0.71399	eval-auc:0.63422


In [30]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.71, Gini: 0.43, KS: 0.32
Eval AUC: 0.63, Gini: 0.27, KS: 0.33
Test AUC: 0.58, Gini: 0.16, KS: 0.13
OOT AUC: 0.62, Gini: 0.25, KS: 0.21


In [31]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.71, Gini: 0.4275, KS: 0.3194
Test AUC: 0.58, Gini: 0.1598, KS: 0.1283
OOT AUC: 0.62, Gini: 0.2491, KS: 0.2123


In [32]:
#Save the Model
xgb_model.save_model('LeoPayu_BA_only_60_in_15mob.json')
print('Model Saved')

Model Saved


In [33]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                     Feature  Importance
13                       cash_withdrawal_sum        23.0
18   diff_btw_stated_salary_vs_actual_salary        23.0
2                  min_balance_last_1_months        22.0
9                   Min_Balance_Min_M1_M2_M3        20.0
15               ratio_active_loan_vs_salary        18.0
6                 bounced_transactions_count        15.0
3                             min_balance_M1        15.0
17  ratio_balance_after_7_days_max_credit_M2        13.0
4                         balance_5th_day_M2        12.0
8                   days_balance_lt_10000_M1        11.0
16  ratio_balance_after_2_days_max_credit_M1        10.0
0            avg_daily_balance_last_1_months         8.0
19    ratio_5th_day_balance_to_max_credit_M3         7.0
12      ratio_total_debit_to_total_credit_M3         6.0
1                             max_balance_M1         5.0
11                   Max_Credit_Avg_M1_M2_M3         4.0
5                         balan

In [34]:
feature_importance_df

Feature  Importance
13                       cash_withdrawal_sum        23.0
18   diff_btw_stated_salary_vs_actual_salary        23.0
2                  min_balance_last_1_months        22.0
9                   Min_Balance_Min_M1_M2_M3        20.0
15               ratio_active_loan_vs_salary        18.0
6                 bounced_transactions_count        15.0
3                             min_balance_M1        15.0
17  ratio_balance_after_7_days_max_credit_M2        13.0
4                         balance_5th_day_M2        12.0
8                   days_balance_lt_10000_M1        11.0
16  ratio_balance_after_2_days_max_credit_M1        10.0
0            avg_daily_balance_last_1_months         8.0
19    ratio_5th_day_balance_to_max_credit_M3         7.0
12      ratio_total_debit_to_total_credit_M3         6.0
1                             max_balance_M1         5.0
11                   Max_Credit_Avg_M1_M2_M3         4.0
5                         balance_3th_day_M1         4.0
14        num_debit_card_trans_last_2_months         3.0
7                    days_balance_lt_5000_M1         2.0
10                           upi_trans_count         1.0

In [35]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.289216         0.392703        0.426309        0.376046    204   
9          0.176471         0.364576        0.376012        0.353579    204   
8          0.151961         0.346844        0.353549        0.340900    204   
7          0.117647         0.334889        0.340800        0.329436    204   
6          0.102941         0.324186        0.329410        0.319244    204   
5          0.088235         0.314730        0.319123        0.310235    204   
4          0.058824         0.305371        0.310222        0.300356    204   
3          0.044118         0.295148        0.300336        0.289113    204   
2          0.049020         0.283835        0.289103        0.278366    204   
1          0.014706         0.272491        0.278250        0.263845    204   

        label_good  label_bad  
Decile                         
10             145         59  
9              168         36  
8              173         31  
7              180         24  
6              183         21  
5              186         18  
4              192         12  
3              195          9  
2              194         10  
1              201          3

In [36]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.182353         0.391211        0.434130        0.373446    170   
9          0.111765         0.364396        0.373382        0.355976    170   
8          0.129412         0.349005        0.355963        0.342718    170   
7          0.105882         0.335660        0.342716        0.329386    170   
6          0.100000         0.323970        0.329299        0.318955    170   
5          0.094118         0.314979        0.318954        0.311187    170   
4          0.082353         0.306851        0.311145        0.302327    170   
3          0.117647         0.296637        0.302296        0.290093    170   
2          0.082353         0.285420        0.290088        0.280658    170   
1          0.046784         0.273611        0.280483        0.265072    171   

        label_good  label_bad  
Decile                         
10             139         31  
9              151         19  
8              148         22  
7              152         18  
6              153         17  
5              154         16  
4              156         14  
3              150         20  
2              156         14  
1              163          8

In [37]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.188406         0.395099        0.423275        0.377102     69   
9          0.173913         0.368885        0.376865        0.360936     69   
8          0.130435         0.352778        0.360887        0.344809     69   
7          0.101449         0.338573        0.344189        0.333078     69   
6          0.147059         0.326792        0.333066        0.321321     68   
5          0.086957         0.316770        0.321260        0.312992     69   
4          0.086957         0.307906        0.312408        0.304035     69   
3          0.057971         0.299214        0.303709        0.292707     69   
2          0.086957         0.286339        0.292412        0.280139     69   
1          0.043478         0.273547        0.279905        0.266023     69   

        label_good  label_bad  
Decile                         
10              56         13  
9               57         12  
8               60          9  
7               62          7  
6               58         10  
5               63          6  
4               63          6  
3               65          4  
2               63          6  
1               66          3